[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/7_tei.ipynb)

# Digitální edice ve standardu TEI-XML


**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

V této kapitole se budeme věnovat práci s digitálními editacemi vytvořenými podle standardu [TEI-XML](https://tei-c.org). TEI-XML je způsob, jak digitalizované literární dílo veřejně prezentovat tak, aby z něj zejména badatelé a studenti z oblasti humanitních věd mohli získávat hodnotné informace například skrz pokročilé prohlédávání. Za tímto účelem jsou texty v těchto edicích ve větší či menší míře buď ručně nebo polautomaticky anotované. Tím se myslí, že jsou v těchto textech označeny specifické výrazy, jako jsou názvy osob či organizací, místní názvy nebo jiné důležité termíny. Stejně tak jsou v záhlaví těchto dokumentů standardizovaným způsobem uvedena nejrůznější metadata, typicky data o autorovi daného textu, jeho vydavateli apod. Projektů, které dnes využívají tento standard, je obrovské množství. Rozsáhlý (avšak zdaleka ne úplný) výčet lze nalézt [zde](https://tei-c.org/Activities/Projects/). 

Důkladné obeznámení se s těmito standardy by vydalo na celý samostatný kurz. Zde však budeme postupovat obdobně jako v jiných kapitolách a pouze si uvnitř prostředí Jupyter notebooku prohlédneme, jak některé takovéto digitalizované edice vypadají a jak s nimi můžeme pracovat.

TEI-XML je spojení dvou akronymů: TEI = Text Encoding Initiative a XML = Extensible Markup Language.
 
**XML** je definováno jako *rozšiřitelný značkovací jazyk* a stejnojmenný formát souborů využívaný zejména pro výměnu dat mezi aplikacemi a pro publikování dokumentů (viz [wikipedia](https://cs.wikipedia.org/wiki/Extensible_Markup_Language). V mnoha ohledech se běžné XML dokumenty podobají kódu HTML stránek (jak uvidíme néže, pro práci s nimi se v rámci Pythonu i využívají stejné nástroje). Na rozdíl od HTML stránek však data naformátovaná za využití XML standardu nejsou určena pro webové prohlížeče.

XML soubor sestává z prvků (elementů), které jsou vyznačeny *tagy*, neboli značkami. Tyto tagy jsou vyznačeny pomocí symbolů `<` a `>`, jsou definovány tvůrcem a tvůrce si může vytvářet zcela vlastní tagy. Vždy zde však musí být právě jeden kořenový prvek, tzv. root. V XML je kořenový prvek jedinečný a nemá svůj vlastní obsah, ale pouze obaluje všechny další prvky v dokumentu. 

Tagy mohou a často obsahují atributy, které definují dodatečné vlastnosti příslušných prvků nebo data provazují s dalšími daty, často pomocí hypertextových odkazů. Na rozdíl od HTML musí být v rámci XML všechny tagy tzv. párové. Tag v podobě `<element1>` prvek otvírá a v podobě `</element>` jej uzavírá. Vše mezi tím je obsahem daného prvku. Prvky se do sebe často vnořují, čímž dokument získává určitou stromovou a navigovatelnou strukturu. 

Nejlépe bude struktura xml souboru vidět na následujícím minimalistickém příkladu:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<root>
    <element1 attribute1="value1">Text for element1</element1>
    <element2 attribute2="value2">Text for element2</element2>
</root>
```
Inciativa **TEI** přichází se standardy pro to, jak tento formát dat použít pro zveřejňování dokumentů, jako jsou literární díla, básně, rozhovory, dopisy, nápisy, papyry a mnoho dalšího (existuje např. i verze pro obrazová díla či hudební notaci). TEI byla ustavena v roce 1987 s cílem "vyvíjet, udžovat a propagovat metody pro kódování humanitních dat v elektronické podobě nezávislé na konkrétním hardweru a softwaru ("to develop, maintain, and promulgate hardware- and software-independent methods for encoding humanities data in electronic form" - viz [zde](https://tei-c.org/about/history/)).
 
Veškerý text daného dokumentu se nachází uvnitř prvku `<body></body>`. Ten se může standardizovaným způsoben dále dělit na kapitoly, verše či řádky, či průpovědi jednotlivých mluvčích, v závislosti na žánru daného dokumentu (všechny tyto možnosti jsou detailně popsány v Guidelines [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/index.html)). Stejně tak jsou jasně nadefinovány způsoby, jak v rámci textu vyznačit výskyty jmenných entit, jako jsou jména osob či místní názvy, nebo například datumy (viz [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/ND.html). Tyto standardy si ukážeme na konkrétníh příkladech níže. Pro detailnější obeznámení pak doporučiji například online kurz [zde](https://teach.dariah.eu/course/view.php?id=32). Praktický úvodní text v češtině je dostupný [zde](https://digital-humanities.phil.muni.cz/clanky/metadata-co-ma-spolecneho-netflix-a-digitalni-humanitni-vedy).

## Cvičení

Jak již bylo uvedeno výše, XML dokumenty mají v mnohém obdobné vlastnosti jako webové stránky ve formátu HTML a k práci s nimi lze použít podobné nástroje. Ač TEI-XML dokumenty nejsou určeny pro webové prohlížeče, lze je často nalézt přímo na webu, např. Nezříka jsou také zpřístupňovány pomocí API. V následujícím civčení proto budeme používat v podstatě stejné nástroje jako v případě kapitol o HTTP a websrapingu a využívání API, zejména knihovny `requests` a `BeautifulSoup`.

In [1]:
import requests
from bs4 import BeautifulSoup

### Korpus německé poezie

Jako první příklad vybereme báseň z korpusu německé poezie **Deutsches Lyrik Korpus** (DLK), dostupného na platformě GitHub [zde](https://github.com/tnhaider/DLK). Jednotlivé básně naformátované ve formě TEI-XML souborů se nachází v této podsložce čítající více než 60,000 položek: https://github.com/tnhaider/DLK/tree/master/DLK/tei/tei_plain.

Vybereme náhodný soubor, otevřeme jej, a zobrazíme jej v "raw" podobě (v oknu souboru klikneme v pravém horním rohu). URL adresu souboru v syrové podobě si zkopírujeme a vložíme jako hodnotu proměnné `url` v buňce
 níže. (Tato url adresa se skládá z informace u umístění souboru v rámci Githubu, která je pro všechny soubory shodná a samotného názvu souboru.)




In [2]:
url = "https://raw.githubusercontent.com/tnhaider/DLK/master/DLK/tei/tei_plain/dta.poem.10006-Zachariae%2C_Justus_Friedrich_Wilhelm-Die_Vergnügungen_der_Melancholey..tei.xml"
resp = requests.get(url)
resp

<Response [200]>

Načtený soubor (v proměnné `resp`) následně parsujeme pomocí knihovny `BeuatifulSoup` do proměnné `soup`, pomocí čehož získáme přístup k stromové struktuře daných dat. Obsah si vypíšeme.

In [3]:
soup = BeautifulSoup(resp.text)
soup

<html><body><tei xmlns="http://www.tei-c.org/ns/1.0">
<teiheader>
<filedesc>
<titlestmt>
<title type="main">Die Vergnügungen  
  der Melancholey.</title>
<author>
<persname>
<surname>Zachariae</surname>
<forename>Justus Friedrich Wilhelm</forename>
</persname>
</author>
<editor>
<persname>
<surname>Haider</surname>
<forename>Thomas Nikolaus</forename>
</persname>
<orgname>University of Göttingen/Max Planck Institute for Empirical Aesthetics, Frankfurt am Main</orgname>
<email>mail@thomashaider.net</email>
</editor>
</titlestmt>
<editionstmt>
<edition>
<name>Deutsches Lyrik Korpus Edition (DLK) v6</name>
<date>9-11-2022</date>
</edition>
</editionstmt>
<extent>
<measure type="stanzas">1</measure>
<measure type="verses">22</measure>
<measure type="verses_per_stanza">1-22</measure>
<measure type="syllables">290</measure>
<measure type="tokens">179</measure>
<measure type="characters">1101</measure>
</extent>
<publicationstmt>
<publisher>
<name></name>
</publisher>
<pubplace></pubplace>
<d

Vidíme kód s jednotlivými prvky. V samotném úvodu vidíme prvky definující metadata o dané básni: jméno autora uvnitř prvku `<persname>`, do sebe vnořuje ještě dva podprvky `<surname>` a `<lastname>`. Všechna tato metadata se nacházejí uvnitř prvku `<teiheader>`. Po jeho uzavření následuje prvek `<text>`, v níž se nachází text příslušné básně. 

Nejprve z textu vyzískejme základní metadata. Stačí nám pohybovat se po stromové struktuře stejně, jako jsme to činili v kapitole o webscrapingu. Například k názvu literárního díla se dostaneme takto:

In [4]:
soup.teiheader.title

<title type="main">Die Vergnügungen  
  der Melancholey.</title>

V tomto případě však vidíme i tagy se všemi jejich atributy. Chceme-li samotný textový obsah, musíme použít `text`. Hodnotu si uložíme do proměnné a vypíšeme si ji.




In [5]:
title = soup.teiheader.title.text
print(title)

Die Vergnügungen  
  der Melancholey.


Stejně můžeme postupovat v případě dalších prvků:

In [6]:
author_surname = soup.teiheader.author.surname.text
author_forename = soup.teiheader.author.forename.text

Vypišme si nyní celé jméno autora a příslušné básně.

In [7]:
print(author_forename, author_surname, "-", title)

Justus Friedrich Wilhelm Zachariae - Die Vergnügungen  
  der Melancholey.


Pozor, že některé tagy se zde mohou objevovat opakovaně. Pomocí metody výše jsme si vypsali pouze první výskyt daného tagu. Chceme-li vypsat všechny výskyty, musíme na objekt `soup` použít metodu `findall()`. Vyzkoušíme s prvkem `surname`.

In [8]:
soup.find_all("surname")

[<surname>Zachariae</surname>, <surname>Haider</surname>]

V případě některých básní budeme mít jmen víc. Je to proto, že je zde ještě příjmení editora dané publikace. Proto jsme výše byli konkrétnější a specifikovali, že se v rámci objektu `soup` pohybujeme v prvku autor.

Nyní se přesuňme k textu samotné básně. Ta je v rámci objektu `soup` dostupná v prvku `text`. V nejsyrovější podobě se k textu básně dostaneme takto.

In [9]:
soup.find("text").get_text() # zde je název tagu "<text>" poněkud matoucí, neboť BeautifulSoup pracuje se stejnojmenným atributem

'\n\n\n          Die Vergnügungen  \n  der Melancholey.\n          \nAndre mögen am lächelnden Abend des Sommers\nWenn sie am dumpfen Geräusch des murmelnden Ba-\nOder das sanftere Roth des streifichten Westens be-\nMich ergetzt nur Nebel und Dunkel des blassen De-\nWenn die Schatten sich dann des langen Abends ge-\nUnd ein schimmernder Stral der matten sterbenden\nDurch den dämmernden Raum, sich bricht: dann laß\nVon dem Jauchzen des Unsinns, das ietzo mit festlichem\nDurch die erleuchteten Zimmer ertönt, dann laß mich\nSitzen, allein nur vergnügt an der niedern klagenden\nSchlummer erweckendem Lied; und laß mich mit mei-\nIn mich gekehrt, den Wechsel der Dinge, die leeren\nUnd die vergebliche Mühe betrachten, die unsrer Er-\nForschen vereitelt, so wie wir die Wüste des Lebens\nDiese gesegnete Stunde der Stille wird alles das Lä-\nSchimmernder Thorheit entdecken, das, gleich des li-\nFalscher zaubrischer Kunst, die allzusicheren Augen\nMit der verborgnen Verblendung getäuscht; den bez

V rámci digitální edice je však báseň dostupná po jednotlivých verších. Pro jednotlivé verše se používá tag `<l>`. Vytvořme si uspořádaný seznam všech veršů:

In [10]:
verses = [l.text for l in soup.find("text").find_all("l")]
verses

['Andre mögen am lächelnden Abend des Sommers',
 'Wenn sie am dumpfen Geräusch des murmelnden Ba-',
 'Oder das sanftere Roth des streifichten Westens be-',
 'Mich ergetzt nur Nebel und Dunkel des blassen De-',
 'Wenn die Schatten sich dann des langen Abends ge-',
 'Und ein schimmernder Stral der matten sterbenden',
 'Durch den dämmernden Raum, sich bricht: dann laß',
 'Von dem Jauchzen des Unsinns, das ietzo mit festlichem',
 'Durch die erleuchteten Zimmer ertönt, dann laß mich',
 'Sitzen, allein nur vergnügt an der niedern klagenden',
 'Schlummer erweckendem Lied; und laß mich mit mei-',
 'In mich gekehrt, den Wechsel der Dinge, die leeren',
 'Und die vergebliche Mühe betrachten, die unsrer Er-',
 'Forschen vereitelt, so wie wir die Wüste des Lebens',
 'Diese gesegnete Stunde der Stille wird alles das Lä-',
 'Schimmernder Thorheit entdecken, das, gleich des li-',
 'Falscher zaubrischer Kunst, die allzusicheren Augen',
 'Mit der verborgnen Verblendung getäuscht; den bezau-',
 'Uns zu t

### Dokumenty EHRI

Jako další příklad použijeme digitální edice vydané projektem [EHRI](https://www.ehri-project.eu) (=The European Holocaust Research Infrastructure). EHRI sestává z mnoha podprojektů. Jedním z nich jsou přepisy raných svědectví přeživších holocaustu ([Early Holocaust Testimony](https://early-testimony.ehri-project.eu)). Tyto edice jsou velice důkladně označkovány ohledně vystujících se jmen osob, místních názvů i organizací, jimž jsou přiřazeny jednoznačné identifikátory, které jsou spravovány v samostatných databázích (vocabularies). Jedná se o vzorový příklad tzv. [Linked Open Data](https://www.ontotext.com/knowledgehub/fundamentals/linked-data-linked-open-data/).    

Vyjdeme  z rozcestníku jednotlivých svědectví dostupného [zde](https://early-testimony.ehri-project.eu/search). Digitální edice ve formátu XML je u každého svědectví dostupná ke stažení přes tlačítko v pravém horním rohu. My však nemusíme soubor fyzicky stahovat, pouze si zkopírujeme URL adresu, na kterou tlačítko pro stažení XML směřuje. 

In [11]:
url = "https://ehri-editions-et.s3.eu-central-1.amazonaws.com/original/34df1a28893a51d05d5909fb3aa99725.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

Jedná se již o delší dokument. Pro představu o jeho podobě si vypíšeme jeho hlavní části samostatně. Nejprve metadata:

In [12]:
print(soup.teiHeader.fileDesc)

<fileDesc>
<titleStmt>
<title>Alexander Schmiedt, the experiences from Kaufering and Dachau</title>
</titleStmt>
<editionStmt>
<edition>EHRI Edition of Early Holocaust Testimony</edition>
<author>
<persName>Michal Frankl</persName>
</author>
<author>
<persName>Magdalena Sedlická</persName>
</author>
<author>
<persName>Wolfgang Schellenbacher</persName>
</author>
</editionStmt>
<publicationStmt>
<publisher>
<ref target="https://www.ehri-project.eu/">European Holocaust Research
            Infrastructure</ref>
</publisher>
<availability>
<licence target="http://creativecommons.org/licenses/by-sa/4.0/">Attribution-ShareAlike
            4.0 International</licence>
</availability>
</publicationStmt>
<sourceDesc>
<msDesc>
<msIdentifier>
<country ref="https://portal.ehri-project.eu/countries/cz">Czech Republic</country>
<repository ref="https://portal.ehri-project.eu/institutions/cz-002279">Jewish Museum in
              Prague</repository>
<collection ref="https://portal.ehri-project.eu/uni

Vidíme tagy s obdobnými názvy jako u dokumentu výše. Popis je však detailnější. Vidíme zde např. i seznamy osob či místních názvů, které se v dokumentu objevují. Nyní se podívejme na samotný textový obsah. Uložme si tuto část stromu do proměnné `textbody`.

In [13]:
textbody = soup.find("text")
textbody

<text>
<body>
<pb facs="EHRI-ET-JMP001_01.jpg" n="1"/>
<p>In <placeName ref="http://www.geonames.org/3067696/prague.html">Prague</placeName>, <date when="1921-08-24">24. VIII. 1921</date></p>
<p>Statement</p>
<p><persName ref="#ehri_et_schmiedt_alexander">Alex. <hi rend="letter-spacing: 2px">Schmiedt</hi></persName>, born <date when="1921-01-27">27. I. 1921</date>, former <term ref="https://portal.ehri-project.eu/keywords/ehri_terms-483" type="subject">prisoner</term> of the <term ref="https://portal.ehri-project.eu/keywords/ehri_terms-496" type="subject">conc. camps</term>
<placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-225" type="camp">Kaufering</placeName> III, <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-230" type="camp">Türkheim</placeName>, and <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-177" type="camp">Dachau</placeName>. <placeName ref="http://www.geonames.org/3067696/prague.html">Praha
          II.</placeName>, 

V textu vidíme vyznačena jména řady osob.

In [14]:
persnames = textbody.find_all("persName")
persnames

[<persName ref="#ehri_et_schmiedt_alexander">Alex. <hi rend="letter-spacing: 2px">Schmiedt</hi></persName>,
 <persName ref="https://portal.ehri-project.eu/authorities/terezin-victims-person-iti-1613340" type="victim">Meir Lindenbaum</persName>,
 <persName ref="https://portal.ehri-project.eu/authorities/ehri_pers-000181">Himmler</persName>,
 <persName ref="#ehri_et_schmiedt_alexander">Alex. Schmiedt</persName>,
 <persName ref="#ehri_et_kratkova_marta">Marta Kratková</persName>,
 <persName ref="#ehri_et_weinberger_robert">Robert Weinberger</persName>,
 <persName ref="#ehri_et_shek_zeev">Zeev
           Scheck</persName>,
 <persName ref="#ehri_et_schmiedt_alexander">Alex
           Schmiedt</persName>]

U některých osob vidíme pod atributem "ref" buď URL adresy do dalších autoritativních databází nebo hashtagem vyznačené identifikátory.

In [15]:
[pers.get("ref") for pers in persnames]

['#ehri_et_schmiedt_alexander',
 'https://portal.ehri-project.eu/authorities/terezin-victims-person-iti-1613340',
 'https://portal.ehri-project.eu/authorities/ehri_pers-000181',
 '#ehri_et_schmiedt_alexander',
 '#ehri_et_kratkova_marta',
 '#ehri_et_weinberger_robert',
 '#ehri_et_shek_zeev',
 '#ehri_et_schmiedt_alexander']

Obdobně získáme seznam místních názvů:

In [16]:
placenames = textbody.find_all("placeName")
placenames

[<placeName ref="http://www.geonames.org/3067696/prague.html">Prague</placeName>,
 <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-225" type="camp">Kaufering</placeName>,
 <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-230" type="camp">Türkheim</placeName>,
 <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-177" type="camp">Dachau</placeName>,
 <placeName ref="http://www.geonames.org/3067696/prague.html">Praha
           II.</placeName>,
 <placeName ref="http://www.geonames.org/3067696/prague.html">Biskupská</placeName>,
 <placeName ref="https://portal.ehri-project.eu/keywords/ehri_ghettos-449" type="ghetto">Theresienstadt</placeName>,
 <placeName ref="https://portal.ehri-project.eu/keywords/ehri_camps-2" type="camp">Auschwitz</placeName>,
 <placeName ref="http://www.geonames.org/3069011/olomouc.html">Olomouc</placeName>,
 <placeName ref="http://www.geonames.org/2761369/vienna.html">Vienna</placeName>,
 <placeName ref="http://www.

Místa jsou zpravidla klasifikována atributem "typ" na "camp", "ghetto" či např. "subjekt". V případě "camp" (koncentrační tábor) zde nalezneme URL odkaz do databáze koncentračních táborů dostupné [zde](https://portal.ehri-project.eu/vocabularies/ehri_camps). V případě jiných místních jmen je zde odkaz na platformu [GeoNames](https://www.geonames.org), se kterou jsme již pracovali v kapitole o websrapingu.    

V některých dokumentech se objevují např. mnohé datumy. V syrovém textu jsou naformátovaná nejrůznějším způsobem, ale v rámci atributu "when" uvnitř tagu jsou dostupná ve strojově čitelné podobě.

In [17]:
dates = soup.find("text").find_all("date")
dates

[<date when="1921-08-24">24. VIII. 1921</date>,
 <date when="1921-01-27">27. I. 1921</date>,
 <date when="1942-07">July 1942</date>,
 <date when="1944-09">September
           1944</date>,
 <date when="1944-10-05">October
           5th</date>,
 <date when="1944-11">November</date>,
 <date when="1945-03-08">March 8th</date>,
 <date when="1945-03">March</date>,
 <date when="1945-04-28">April 28th</date>,
 <date when="1945-04-29">April 29th</date>,
 <date when="1945-08-21">21. VIII. 1945</date>]

In [18]:
# datumy ve strojově čitelné podobě:
[date.get("when") for date in dates]

['1921-08-24',
 '1921-01-27',
 '1942-07',
 '1944-09',
 '1944-10-05',
 '1944-11',
 '1945-03-08',
 '1945-03',
 '1945-04-28',
 '1945-04-29',
 '1945-08-21']

Všechny tyto anotace mohou být použity k rozkrývání vztahů mezi dokumenty a nejrůznějším vizualizacím.

### Antická literatura: anglické překlady řeckých textů a řecké originály

Studium antiky obecně, a klasické řecké a latinské literatury zvláště, je dlouhodobě doménou, kde se digitální edice ve formě TEI-XML těší velké oblibě. V současnosti máme takto zpracovanou a volně dostupnou v zásadě veškerou antickou řeckou literaturu, a to díky projektům [Perseus Digital Library](https://www.perseus.tufts.edu/hopper/) a [First1KGreek Project](https://www.perseus.tufts.edu/hopper/), z nichž druhý je pokračováním prvého. Oba projekty průběžně udržují TEI-XML edice jimi zveřejňovaných textů na githubu [zde](https://github.com/PerseusDL/canonical-greekLit) a [zde](https://github.com/OpenGreekAndLatin/First1KGreek). Spolu s řeckými originály jsou v TEI-XML podobě zveřejňovány i anglické překlady.

Podívejme se nyní řecký text Platónova dialogu *Eutyfrón*, patrně jednoho z nejstarších Platónových dialogů, která má relativně přímočarou strukturu: celý text dialogu je tvořen konverzací mezi Sókratem a Eutyfrónem, kteří spolu vedou debatu ohledně podstaty zbožnosti. Text dialogu načteme přímo z Githubu:

In [19]:
url = "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0059/tlg001/tlg0059.tlg001.perseus-eng2.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

Přímá řeč je zachycena pomocí prvku `<said>`, jehož atribut "who" definuje řečníka. Díky tomu si můžeme celý dialog rozdělit snadno na slova pronesená Sókratem a slova pronesena Eutyfrónem, přičemž můžeme očekávat, že Platónova názorová pozice bude patrně spíše zachycena v Sókratových slovech.

In [20]:
said_socrates = [said for said in soup.find_all("said") if said.get("who")=="#Socrates"]
len(said_socrates)

117

In [21]:
# náhodný výběr Sókratových průpovědí
said_socrates[40:45]

[' Then, as it seems, the same things are hated and loved by the gods, and the same things would be dear and hateful to the gods.',
 ' And then the same things would be both holy and unholy, Euthyphro, according to this statement.',
 'it would not be surprising if, in punishing your father as you are doing, you were performing an act that is pleasing to Zeus, but hateful to Cronus and Uranus, and pleasing to Hephaestus, but hateful to Hera, and so forth in respect to the other gods, if any disagree with any other about it.',
 'or had done anything else whatever wrongfully, ought not to pay the penalty?',
 ' Yes, but do they acknowledge, Euthyphro, that they have done wrong and, although they acknowledge it, nevertheless say that they ought not to pay the penalty?']

Samotná slova mluvčího se však nacházejí až za tagem `<label>`. Kód tedy ještě upravíme tak, abychom dostali pouze text průpovědi bez tohoto tagu.

In [22]:
said_socrates = [said.contents[-1] for said in soup.find_all("said") if said.get("who")=="#Socrates"]
said_socrates[40:45]

[' Then, as it seems, the same things are hated and loved by the gods, and the same things would be dear and hateful to the gods.',
 ' And then the same things would be both holy and unholy, Euthyphro, according to this statement.',
 'it would not be surprising if, in punishing your father as you are doing, you were performing an act that is pleasing to Zeus, but hateful to Cronus and Uranus, and pleasing to Hephaestus, but hateful to Hera, and so forth in respect to the other gods, if any disagree with any other about it.',
 'or had done anything else whatever wrongfully, ought not to pay the penalty?',
 ' Yes, but do they acknowledge, Euthyphro, that they have done wrong and, although they acknowledge it, nevertheless say that they ought not to pay the penalty?']

In [23]:
said_eutyphro = [said.contents[-1] for said in soup.find_all("said") if said.get("who")=="#Euthyphro"]
len(said_eutyphro)

116

Může být zajímavé se podívat, jak je v této struktuře zachycen Platónův dialog *Symposion*. Jedná se totiž o dialog nepřímý, vnořený do vcelku komplexního narativního rámce. Dialog je vyprávěný postavou Apollodora v rozhovoru s nejmenovaným společníkem. Hlavní text dialogu je tedy do důsledku vzato přímou řečí, v rámci které se odehrávají další přímé řeči.

In [25]:
url = "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0059/tlg011/tlg0059.tlg011.perseus-eng2.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

In [26]:
len(soup.find_all("said"))

98

In [27]:
said_apollodorus = [said.contents[-1].text for said in soup.find_all("said") if said.get("who")=="#Apollodorus"]
said_apollodorus[:5]

['You must know it is many a year that Agathon has been away from home and country, and not yet three years that I have been consorting with Socrates and making it my daily care to know whatever he says or does.',
 ' From your point of view, I daresay, I seem a hapless creature, and I think your thought is true.  I, however, do not think it of you:  I know it for sure.',
 ' \n         My dear sir, obviously it must be a mere crazy aberration in me, to hold this opinion of myself and of you all!',
 ' just as my friend told it to me.',
 'but where is the man?']

Uvnitř těchto průpovědí se pak nachází text dialogu samotného, sestávající především z jednotlivých řečí na téma boha Eróta. Takto například získáme Faidrovu oslavnou řeč:

In [28]:
[said.text for said in soup.find_all("said") if said.get("who")=="#Phaedrus"]

['Of the most venerable are the honors of this god, and the proof of it is this:  parents of Love there are none, nor are any recorded in either prose or verse.  Hesiod says that Chaos came first into being—and thereafter roseBroad-breasted Earth, sure seat of all for aye,And Love.Hes. Theog. 116\n     Acusilaus An Argive compiler of genealogies in the first part of the fifth century B.C. also agrees with Hesiod, saying that after Chaos were born these two, Earth and Love.  Parmenides says of Birth that she invented Love before all other gods. Parmenides fr. 132; Cf. Aristot. Met. 1.984b. \nThus Love is by various authorities allowed to be of most venerable standing; and as most venerable, he is the cause of all our highest blessings.  I for my part am at a loss to say what greater blessing a man can have in earliest youth than an honorable lover, or a lover than an honorable favorite.  For the guiding principle we should choose for all our days, if we are minded to live a comely life,

Podíváme-li se na příslušné místo před začátkem této řeči, můžeme zaznamenat, že Faidrova řeč začíná, aniž by byla průvodní Apollodorova řeč ukončena: 
```xml
<p><said who="#Apollodorus" rend="merge">
<milestone ed="P" unit="para"/>First then, as I said, he told me that the speech of Phaedrus began with points of this sort—that Love was a great god, among men and gods a marvel; and this appeared in many ways, but notably in his birth. <milestone unit="section" resp="Stephanus" n="178b"/>
<said who="#Phaedrus" direct="false">Of the most venerable are the honors of this god, and the proof of it is this:  parents of Love there are none, nor are any recorded in either prose or verse...
```

Také si zde povšimněme tagu `<milestone>` a jeho atributů, které vyznačují stránkování v rámci kritické edice, které standardně používáme i při citování z českého překladu.

### Antická literatura: řecké originály

Digitální edice řeckých originálů se v GitHub repozitoři projektu Perseus nachází ve stejných složkách jako výše použité anglické překlady. Tj. v případě dialogu *Eutyfrón: https://github.com/PerseusDL/canonical-greekLit/tree/master/data/tlg0059/tlg001. Všimněme si v této url adrese kódů "tlg0059" a "tlg001". "tlg0059" je standardní identifikátor Platóna napříč různými korpusu, a "tlg001" je taktéž standardní identifikátor dialogu *Eutyfrón* v rámci Platónových děl. Tyto identifikátory proto můžeme použít i jinde. Využijeme zde rozsáhlý automaticky anotovaný korpus řeckých antických textů zvaný GLAUx (k tomu viz [tento](https://aclanthology.org/2021.lchange-1.6/) průvodní text). 

Velká část klasických řeckých textů je nyní dostupná i ve formátu, který jde z jazykového hlediska více do detailu a nabízí texty důkladně morfologicky anotované (tzv. *treebanks* [[wikipedia](https://en.wikipedia.org/wiki/Treebank)]). Tím se v zásadě myslí, že jednotlivá slova jsou zde opatřena svým gramatickým určením a příslušný výraz je zde uveden i v základním tvaru, tak jak jej najdeme ve slovníku (např. veškerá podstatná jména jsou v nominativu singuláru apod.). Nejedná si již v pravém smyslu o digitální edice, neboť zde není věnována pozornost obecným metadatům, která lze snadno dohledat jinde. Ale technologie je v zásadě stejná jako výše.

Podívejme se v této podobě opět na dialog Eutyfrón (povšimněme si číselného kódu v názvu souboru). Omezíme se však pouze na několik prvních vět samotného textu.

In [29]:
url = "https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/public/xml/0059-001.xml"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="xml")

V této podobě budou naši výchozí jednotkou jednotlivé věty. Ty z dokumentu snadno extrahujeme pomocí vyhledání všech výskytů tagu  `<sentences>`. Obratem si vypíšeme 3 jeho první výskyty.

In [30]:
sents = soup.find_all("sentence")
sents[:3]

[<sentence document_id="0059-001" id="1" subdoc="2">
 <word form="τί" head="5794848" id="5794842" lemma="τίς" postag="a-s---na-" relation="ADV" speaker="#Εὐθύφρων"/>
 <word form="νεώτερον" head="5794842" id="5794843" lemma="νέος" postag="a-s---nac" relation="ATR" speaker="#Εὐθύφρων"/>
 <word form="," head="5794842" id="5794844" lemma="," postag="u--------" relation="PUNCT" speaker="#Εὐθύφρων"/>
 <word form="ὦ" head="5794846" id="5794845" lemma="ὦ" postag="i--------" relation="AuxZ" speaker="#Εὐθύφρων"/>
 <word form="Σώκρατες" head="5794848" id="5794846" lemma="Σωκράτης" postag="n-s---mv-" relation="ExD" speaker="#Εὐθύφρων"/>
 <word form="," head="5794842" id="5794847" lemma="," postag="u--------" relation="PUNCT" speaker="#Εὐθύφρων"/>
 <word form="γέγονεν" head="0" id="5794848" lemma="γίγνομαι" postag="v3sria---" relation="PRED" speaker="#Εὐθύφρων"/>
 <word form="," head="5794842" id="5794849" lemma="," postag="u--------" relation="PUNCT" speaker="#Εὐθύφρων"/>
 <word form="ὅτι" head="5

Textová data jsou zde zaznamenána tak, že každá věta je zde rozdělena na jednotlivá slova, kterému odpovídá prvek `<word>` v jehož atributech lze nalézt jak slovo v podobě, jak jej najdeme v samotném textu (hodnota atributu "form"), tak ve slovníkové podobě, tzv. "lemma"). Atribut "postag" kóduje tzv. značkování části řeči ("part-of-speech") a atribut "speaker" informaci o mluvčím, tak jak jsme se s ní setkali výše. Všechny tyto atributy jsou velice užitečné a mohou být použity pro nejrůznější jazykové analýzy daných textů, jak z hlediska obsahu, tak stylu či formy. 

Ukážeme si jeden příklad: První písmeno atributu "postag" definuje slovní druh a písmeno "n" značí podstatné jméno ("noun"). Když toto víme, tak pomocí kódu níže z daného textu vypíšeme lemmata všech podstatných jmen, které se v něm vyskytují. 

In [31]:
nouns = [word["lemma"] for word in soup.find_all("word") if word["postag"][0]=="n"]
nouns[:10]

['Σωκράτης',
 'Λύκειον',
 'διατριβή',
 'βασιλεύς',
 'στοά',
 'δίκη',
 'βασιλεύς',
 'Εὐθύφρων',
 'δίκη',
 'γραφή']

Slova v této podobě bychom nyní mohli podrobit nejrůznějším kvantitativním analýzám. To si však necháme do samostatné kapitoly, kde si tyto metody ozkoušíme na textových datech v češtině, která však do obdobné podoby budeme zpracovávat jinými způsoby než tak, že bychom je přímo extrahovali z již zveřejněného korpusu.

### Britská poezie 19. století

Pokud nás zajímá poezie, můžeme si samostatně vyzkoušet práci s velice důkladnými digitálními edicemi britské poezie dostupnými [zde](https://www.eighteenthcenturypoetry.org/authors/). Postup bude obdobný jako v předchozích příkladech. 